In [1]:
# Add Lincoln to system path
import sys
sys.path.append("/Users/seth/development/lincoln/")

In [2]:
from torch import Tensor
import torch
import numpy as np
from numpy import random

import typing
from typing import List, Tuple

# Demo

Building a simple classifier on the breast cancer dataset.

## Data prep

In [3]:
from sklearn.datasets import load_breast_cancer
breast_cancer = load_breast_cancer()
features = breast_cancer.data
labels = breast_cancer.target
feature_names = breast_cancer.feature_names

/Users/seth/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/seth/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [4]:
from lincoln.utils import standardize

features = standardize(features)

In [5]:
import lincoln as lnc
from lincoln.layers import Dense, NeuralNetwork
from lincoln.losses import LogSigmoidLoss, MeanSquaredError, LogLoss, Loss
from lincoln.optimizers import SGD
from lincoln.layers import Layer
from lincoln.activations import Sigmoid, LogSigmoid
from lincoln.models import Logistic

### `Logistic` model 

In [6]:
# network = NeuralNetwork([
#     Dense(1, activation=Sigmoid()),
# ])

# model = Logistic(network,
#                  loss=MeanSquaredError(), 
#                  optimizer=SGD(0.1))

# model.fit(features, labels, batch_size=128, log_ps=False,
#           epochs=100, print_every=10)

Something is off...

In [7]:
# network = lnc.Sequential(
#             lnc.layers.Dense(10),
#             lnc.layers.Dense(1, activation=LogSigmoid()),
#             )
# model = Logistic(network,
#                  loss=LogSigmoidLoss(network), 
#                  optimizer=SGD(0.003))
# model.fit(features, labels, batch_size=128, log_ps=True,
#           epochs=500, print_every=100)

**TODO:** `print_every` not working.

## Regression network

In [8]:
from sklearn.datasets import load_boston

boston = load_boston()

data = boston.data
target = boston.target
features = boston.feature_names

from sklearn.preprocessing import StandardScaler
s = StandardScaler()
data = s.fit_transform(data)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.3, random_state=80718)

X_train, X_test, y_train, y_test = Tensor(X_train), Tensor(X_test), Tensor(y_train), Tensor(y_test)

/Users/seth/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


## Custom `Model` class and `train` function

In [9]:
from lincoln.losses import MeanSquaredError
from lincoln.optimizers import SGD
from lincoln.utils import permute_data, generate_batch, to_2d

from lincoln.activations import LinearAct

In [10]:
class LinearModel(object):
    def __init__(self, 
                 network: NeuralNetwork):
        self.network = network

    def train(self, 
              X_train: Tensor, 
              y_train: Tensor,
              X_test: Tensor = None,
              y_test: Tensor = None,
              iterations: int=500, print_every: int=100, 
              batch_size: int=32, seed: int = 1)-> None:
 
        torch.manual_seed(seed)
        y_train, y_test = to_2d(y_train, "col"), to_2d(y_test, "col")
        start = 0

        for i in range(iterations):
                 
            # Generate batch
            if start >= X_train.shape[0]:
                X_train, y_train = permute_data(X_train, y_train)
                start = 0

            X_batch, y_batch = generate_batch(X_train, 
                                              y_train, 
                                              start, batch_size)

            self.network.train_batch(X_batch, y_batch)
            
            if i % print_every == 0:
                loss = self.network.forward_loss(X_test, y_test)
                print("Loss: ", loss) 

In [11]:
lr = NeuralNetwork(
    layers=[Dense(neurons=1, 
                  activation=LinearAct())], 
            learning_rate=0.0002,
            loss = MeanSquaredError())


model = LinearModel(network=lr)

In [12]:
num_iter = 100
print_every = 10

model.train(X_train, y_train, X_test, y_test, 
      iterations = num_iter, 
      print_every = print_every, 
      batch_size = 23, 
      seed=80718);

Loss:  tensor(82850.5078)
Loss:  tensor(67651.3516)
Loss:  tensor(57402.3750)
Loss:  tensor(49771.3438)
Loss:  tensor(43768.3203)
Loss:  tensor(38882.3555)
Loss:  tensor(34814.5312)
Loss:  tensor(31374.4102)
Loss:  tensor(28432.3574)
Loss:  tensor(25895.2539)
